# Parsing PDFs 

Pull in all of the PDF files and create objects for the text inside each one. 

This also probably should include a spell check

In [1]:
import string 

In [2]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
import PyPDF2 
from glob import glob

pdfs = glob('../pdfs/*.pdf') 

In [2]:
pdfs

['../pdfs/Cruz - 2010 - Chapter Six. Expanding The View The Challenges Of.pdf',
 '../pdfs/Cruz - 2010 - Index.pdf',
 '../pdfs/Frederiks and Nagy - 2016 - Religion, migration, and identity methodological .pdf',
 '../pdfs/Cruz - 2010 - Introduction.pdf',
 '../pdfs/Cruz - 2010 - Chapter One. Geographies Of Domestication Mapping.pdf',
 '../pdfs/cruz2010.pdf',
 '../pdfs/Cruz - 2010 - Chapter Two. Frontiers Of Struggle Negotiating Fi.pdf',
 '../pdfs/Cruz - 2010 - Conclusion.pdf',
 '../pdfs/Cruz - 2010 - Chapter Four. Exploring Theological Markers Delor.pdf',
 '../pdfs/Cruz - 2010 - An intercultural theology of migration pilgrims i.pdf',
 "../pdfs/Nguyen and Prior - 2014 - God's people on the move biblical and global pers.pdf",
 '../pdfs/Cruz - 2010 - Chapter Three. Expanding The Boundaries Theologic.pdf',
 '../pdfs/Gods People on the Move 2.pdf',
 '../pdfs/Cruz - 2010 - Chapter Five. A Different Cartography Mapping The.pdf',
 '../pdfs/Izuzquiza - 2011 - Breaking bread notes for a political t

# Preparing Texts 

## Lemmatizing and cleaning 

In [3]:
import string
import nltk 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as STOPWORDS 

PUNCDIG_TRANSLATOR = str.maketrans('', '', string.punctuation+string.digits)

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sgoodwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sgoodwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def text_clean(text):
    clean_list = []
    words = nltk.word_tokenize(text)
    for w in words:
        if w not in STOPWORDS:
            w = w.translate(PUNCDIG_TRANSLATOR)
            clean_list.append(lemmatizer.lemmatize(w))
    return clean_list

In [5]:
text_clean("The striped bats are hanging on their feet3 for best.".lower())

['striped', 'bat', 'hanging', 'foot', 'best', '']

## Extracting Texts

In [17]:
pdf = open(pdfs[0], 'rb')
pdf_obj = PyPDF2.PdfFileReader(pdf)


In [18]:
print('No. of pages: {}'.format(pdf_obj.numPages))

No. of pages: 35


In [6]:
def pdf_extractor(pdf, corpus_list, text_list):
    '''Extract the text of pdfs and return a dictionary with
    the file name as a key, and the value being a list of the pages
    and the containing texts
    '''
    pdf_file_obj = open(pdf, 'rb')
    pdf_obj = PyPDF2.PdfFileReader(pdf_file_obj)
    for pn in range(0,pdf_obj.numPages):
        page = pdf_obj.getPage(pn)
        text = page.extractText().lower()
        cleaned_list = text_clean(text)
        corpus_list.append(cleaned_list)
        # corpus_list.append(page.extractText())
        text_list.append((pdf, pn))
        # if you want to create a dictionary
        # text_dict.setdefault(pdf, []).append(page.extractText())
    pdf_file_obj.close()
    return corpus_list, text_list

In [9]:
corpus_list = []
text_list = []

for pdf in pdfs:
    corpus_list, text_list = pdf_extractor(pdf, corpus_list, text_list)

# Creating LDA Model 



In [10]:
from gensim import corpora 
from gensim.models.ldamodel import LdaModel 

In [44]:
text = text_dict['../pdfs/Cruz - 2010 - Chapter Six. Expanding The View The Challenges Of.pdf']

In [11]:
def prepare_topic_model(corpus_list):
    corpus_dict = corpora.Dictionary(corpus_list)
    corpus_dict.filter_extremes(no_below=100, no_above=0.5)
    corpus = [corpus_dict.doc2bow(text) for text in corpus_list]
    lda_model = LdaModel(corpus=corpus, 
                        id2word=corpus_dict, num_topics=25,
                        random_state=100, update_every=1,
                        chunksize=100, passes=50,
                        alpha='symmetric', per_word_topics=True)
    return lda_model, corpus, corpus_dict

In [12]:
lda_model, corpus, corpus_dict = prepare_topic_model(corpus_list)

# Visualizing LDA Model

In [13]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, corpus_dict, mds='mmds')

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [_prepare.py:257]


In [15]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.319844  0.072203       1        1  19.960415
11    -0.151797  0.194509       2        1  12.622334
19    -0.227208  0.160259       3        1   9.017085
6      0.057691 -0.218967       4        1   5.768025
4     -0.252737  0.321251       5        1   4.852766
18     0.125846  0.083278       6        1   4.220944
2      0.303519 -0.319604       7        1   4.094694
15     0.174816 -0.135898       8        1   3.876787
8      0.338184 -0.066354       9        1   3.560840
0      0.349234  0.195141      10        1   3.537691
10    -0.021100  0.367284      11        1   3.452844
21     0.415862  0.076558      12        1   3.449908
9     -0.510196  0.195572      13        1   3.078662
20    -0.304337 -0.245118      14        1   2.901001
12    -0.055579 -0.496234      15        1   2.762663
14    -0.310297 -0.390335      16        1   2.433632
1     -0.127773 -0.319014      17        1   2.259889
22    -0.465623 -0.138044      18        1   1.881467
5      0.187097  0.452513      19        1   1.704855
7      0.169871 -0.477616      20        1   1.371303
17     0.477368 -0.200538      21        1   1.171152
3     -0.150962  0.528542      22        1   1.035468
24     0.388452  0.388497      23        1   0.846288
23    -0.045244 -0.013943      24        1   0.069642
16    -0.045244 -0.013943      25        1   0.069642, topic_info=     Category        Freq          Term        Total  loglift  logprob
term                                                                  
33    Default  2532.00000             ”  2532.000000  30.0000  30.0000
84    Default   932.00000        church   932.000000  29.0000  29.0000
48    Default  1812.00000             ’  1812.000000  28.0000  28.0000
32    Default  2171.00000             “  2171.000000  27.0000  27.0000
31    Default   679.00000             –   679.000000  26.0000  26.0000
15    Default   943.00000     migration   943.000000  25.0000  25.0000
24    Default   704.00000      theology   704.000000  24.0000  24.0000
28    Default  1182.00000         woman  1182.000000  23.0000  23.0000
53    Default   928.00000       migrant   928.000000  22.0000  22.0000
11    Default   690.00000      filipino   690.000000  21.0000  21.0000
21    Default   583.00000      struggle   583.000000  20.0000  20.0000
72    Default  1016.00000           dhs  1016.000000  19.0000  19.0000
43    Default  1019.00000             s  1019.000000  18.0000  18.0000
7     Default   797.00000             e   797.000000  17.0000  17.0000
66    Default   372.00000          work   372.000000  16.0000  16.0000
29    Default   446.00000        worker   446.000000  15.0000  15.0000
9     Default   437.00000        eology   437.000000  14.0000  14.0000
8     Default   416.00000            ed   416.000000  13.0000  13.0000
45    Default   317.00000        social   317.000000  12.0000  12.0000
90    Default   241.00000             d   241.000000  11.0000  11.0000
89    Default   280.00000         right   280.000000  10.0000  10.0000
16    Default   534.00000           new   534.000000   9.0000   9.0000
51    Default   259.00000      economic   259.000000   8.0000   8.0000
63    Default   352.00000      religion   352.000000   7.0000   7.0000
55    Default   491.00000        people   491.000000   6.0000   6.0000
57    Default   394.00000     religious   394.000000   5.0000   5.0000
18    Default   357.00000    philippine   357.000000   4.0000   4.0000
35    Default   435.00000           god   435.000000   3.0000   3.0000
6     Default   333.00000      domestic   333.000000   2.0000   2.0000
105   Default   255.00000     immigrant   255.000000   1.0000   1.0000
...       ...         ...           ...          ...      ...      ...
92    Topic25     0.26324          role   193.390549   0.6702  -4.6634
77    Topic25     0.26324          need   194.287186   0.6655  -4.6634
20    Topic25 

In [17]:
pyLDAvis.save_html(vis, 'PrelimTopicModel.html')